<a href="https://colab.research.google.com/github/joeenn/Kapita-Selekta-Matematika-Terapan/blob/main/Analysis%20Sentiment%20Review%20Film%20Frozen%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install scattertext

     |████████████████████████████████| 7.3 MB 4.6 MB/s 
     |████████████████████████████████| 24.1 MB 2.7 kB/s 
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9310 sha256=30579f21d69ce4b689bddb7080aaabe44a83e3ac0ba201b9f326b98168847b1f
  Stored in directory: /root/.cache/pip/wheels/cb/19/58/4e8fdd0009a7f89dbce3c18fff2e0d0fa201d5cdfd16f113b7
Successfully built flashtext
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
from bs4 import BeautifulSoup
import requests

%matplotlib inline
import scattertext as st
import re, io
from pprint import pprint
import pandas as pd
import numpy as np
from scipy.stats import rankdata, hmean, norm
import spacy
import os, pkgutil, json, urllib
from urllib.request import urlopen
from IPython.display import IFrame
from IPython.core.display import display, HTML
from scattertext import CorpusFromPandas, produce_scattertext_explorer
display(HTML("<style>.container { width:98% !important; }</style>"))

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
page_urls = ['https://www.rottentomatoes.com/m/real_steel/reviews',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=2',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=3',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=4',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=5',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=6',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=7',
             'https://www.rottentomatoes.com/m/real_steel/reviews?type=&sort=&page=8',]

In [ ]:
reviews = []
tomato = []

#iterate through each url in the list.
for url in page_urls:
    response = requests.get(url)
    #check whether the response code is not 200. It means there is error in server response
    if response.status_code != 200:
        continue
    #if the response is 200, we can continue with scrapping the content of the response.
    else:
        html_code = response.content
        #choose 'lxml' as the parser for the application as it more efficient than other parsers.
        soup = BeautifulSoup(html_code, 'lxml')
        #find the html tag with review table as the class.
        review_table = soup.find('div',{'class':'review_table'})
        #iterate through every rows in the review table in the current page.
        for i in review_table.find_all('div',{'class':'critic__review-quote'}):
            #append the review in the list.
            reviews.append(i.get_text(strip=True))
        #iterate through every rows inorder to find the tomato.
        for j in review_table.find_all('div',{'class':'col-xs-16 review_container'}):
            a=j.find('div',{'class':'review_icon icon small fresh'}) or j.find('div',{'class':'review_icon icon small rotten'})
            #append the type in the list.
            tomato.append(a.attrs['class'][-1])

AttributeError: ignored

In [ ]:
import pandas as pd

In [ ]:
reviews = pd.DataFrame(zip(reviews,tomato), columns=['Critics','Rotten_Fresh'])

In [ ]:
reviews

,Critics,Rotten_Fresh
0,The end of Stranger Things S2 slays whatever e...,fresh
1,This was a near perfect season and Stranger Th...,fresh
2,What this season does so well really is that i...,fresh
3,The women are complex and well-rounded but the...,fresh
4,Another compelling season with a story that is...,fresh
...,...,...
145,The show's core cast remains an extremely vers...,fresh
146,"Stranger Things 2 is quite good and, if your e...",fresh
147,"Stranger Things may still be strange, but it's...",fresh
148,A show that was primarily influenced by the St...,fresh


In [ ]:
reviews.Critics[0]

'The end of Stranger Things S2 slays whatever early-season nitpicks existed.'

In [ ]:
reviews.Rotten_Fresh

0      fresh
1      fresh
2      fresh
3      fresh
4      fresh
       ...  
145    fresh
146    fresh
147    fresh
148    fresh
149    fresh
Name: Rotten_Fresh, Length: 150, dtype: object

In [ ]:
reviews['parsed'] = reviews.Critics.apply(nlp)

In [ ]:
reviews

,Critics,Rotten_Fresh,parsed
0,The end of Stranger Things S2 slays whatever e...,fresh,"(The, end, of, Stranger, Things, S2, slays, wh..."
1,This was a near perfect season and Stranger Th...,fresh,"(This, was, a, near, perfect, season, and, Str..."
2,What this season does so well really is that i...,fresh,"(What, this, season, does, so, well, really, i..."
3,The women are complex and well-rounded but the...,fresh,"(The, women, are, complex, and, well, -, round..."
4,Another compelling season with a story that is...,fresh,"(Another, compelling, season, with, a, story, ..."
...,...,...,...
145,The show's core cast remains an extremely vers...,fresh,"(The, show, 's, core, cast, remains, an, extre..."
146,"Stranger Things 2 is quite good and, if your e...",fresh,"(Stranger, Things, 2, is, quite, good, and, ,,..."
147,"Stranger Things may still be strange, but it's...",fresh,"(Stranger, Things, may, still, be, strange, ,,..."
148,A show that was primarily influenced by the St...,fresh,"(A, show, that, was, primarily, influenced, by..."


In [ ]:
print("Document Count")
print(reviews.groupby('Rotten_Fresh')['Critics'].count())
print("Word Count")
reviews.groupby('Rotten_Fresh').apply(lambda x: x.Critics.apply(lambda x: len(x.split())).sum())

Document Count
Rotten_Fresh
fresh     141
rotten      9
Name: Critics, dtype: int64
Word Count


Rotten_Fresh
fresh     3415
rotten     183
dtype: int64

In [ ]:
corpus = st.CorpusFromParsedDocuments(reviews, category_col='Rotten_Fresh', parsed_col='parsed').build()

In [ ]:
html = produce_scattertext_explorer(corpus,category='fresh',category_name='Fresh', not_category_name='Rotten',width_in_pixels=1000, minimum_term_frequency=5)
file_name = 'StrangerThingsSeason02.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [ ]:
html = st.produce_scattertext_explorer(corpus,
                                       category='fresh',
                                       category_name='Fresh',
                                       not_category_name='Rotten',
                                       minimum_term_frequency=5,
                                       width_in_pixels=1000,
                                       transform=st.Scalers.log_scale_standardize)
file_name = 'ST02Log.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)

In [ ]:
html = produce_scattertext_explorer(corpus,
                                    category='fresh',
                                    category_name='Fresh',
                                    not_category_name='Rotten',
                                    width_in_pixels=1000,
                                    minimum_term_frequency=5,
                                    transform=st.Scalers.percentile)
file_name = 'Conventions2012ScattertextRankData.html'
open(file_name, 'wb').write(html.encode('utf-8'))
IFrame(src=file_name, width = 1200, height=700)